# TFRecord
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- Tensorflow에서 제공하는 데이터셋을 저장방식.
    - 데이터 양이 많을 경우 이를 Binary로 Seralization(직렬화)하여 하나의 파일로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer 형태로 Serialization을 수행해서 저장할 수 있는 TFRecords 파일 포맷 형태를 지원한다. 
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecords 파일에 저장할 수 있다.
- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.
- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, byte 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환
- tf.tran.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```
> - **직렬화(Serialization):** 메모리에 저장된 다양한 타입의 값을 디스크(네트워크)에 저장할 수 있는 상태로 변환하는 것.
> - **binary data:** 디스크에 저장되는 0, 1로 구성된 데이터

In [1]:
import tensorflow as tf
import numpy as np 

In [3]:
def _bytes_feature(value):
    """
    bytes string, bytes(이진파일) 타입의 value를 받아서 BytesList로 변환하는 함수
    """
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() #BytesList는 EagerTensor타입은 변환할 수 없기 때문에 ndarray로 변환
    return tf.train.Feature(bytes_list=tf.train.BytesList(value = [value]))    

def _float_feature(value):
    """
    float타입의 value를 받아서 FloatList 변환 하는 함수
    """
    return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))


def _int64_feature(value):
    """
    int, uint, bool 타입의 value를 받아서 Int64List로 변환하는 함수
    """
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))
    pass

In [4]:
v = 30
r = _int64_feature(v)
print(type(r))
r

<class 'tensorflow.core.example.feature_pb2.Feature'>


int64_list {
  value: 30
}

In [5]:
v = 5.73
r = _float_feature(v)
print(type(r))
r

<class 'tensorflow.core.example.feature_pb2.Feature'>


float_list {
  value: 5.730000019073486
}

In [7]:
v = b"abc" #bytes str (문자열 : str객체, byte str : unicode)
_bytes_feature(v)

bytes_list {
  value: "abc"
}

In [8]:
v = 'test'
v2 = v.encode('utf-8')
_bytes_feature(v2)

bytes_list {
  value: "test"
}

In [11]:
with open('capture_images/two-ea04e07e-16a0-11ec-9092-b886876ad90e.jpg', 'rb') as f:
    img = f.read()
type(img), type(v2), type(b'abc')


(bytes, bytes, bytes)

In [12]:
r = _bytes_feature(img)
type(r)

tensorflow.core.example.feature_pb2.Feature

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력하기 전에 변환해야 한다.

In [13]:
f = _float_feature(0.234)
f


float_list {
  value: 0.23399999737739563
}

In [14]:
f.SerializeToString()

b'\x12\x06\n\x04\xb2\x9do>'

# TFRecord 저장 예제

### tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.


In [34]:
N_DATA = 1000 #1000개의 데이터 포인트를 생성

#TFRecord에 저장할 가상의 DataSet을 생성
#bool
feature0 = np.random.choice([False, True], N_DATA)
#int
feature1 = np.random.randint(0,5, N_DATA)
#float
feature2 = np.random.normal(size = N_DATA)
#str
s_list = [b'cat', b'dog', b'tiger']
feature3 = np.random.choice(s_list, N_DATA)

feature0.shape, feature1.shape, feature2.shape, feature3.shape
feature3

array([b'tiger', b'cat', b'tiger', b'tiger', b'cat', b'dog', b'tiger',
       b'dog', b'tiger', b'tiger', b'tiger', b'cat', b'dog', b'tiger',
       b'dog', b'dog', b'cat', b'dog', b'dog', b'cat', b'dog', b'cat',
       b'dog', b'cat', b'cat', b'tiger', b'cat', b'tiger', b'dog',
       b'tiger', b'dog', b'tiger', b'dog', b'tiger', b'dog', b'tiger',
       b'tiger', b'cat', b'tiger', b'tiger', b'tiger', b'cat', b'tiger',
       b'tiger', b'cat', b'dog', b'cat', b'tiger', b'dog', b'dog',
       b'tiger', b'dog', b'tiger', b'tiger', b'dog', b'tiger', b'tiger',
       b'dog', b'cat', b'cat', b'dog', b'cat', b'tiger', b'cat', b'dog',
       b'dog', b'tiger', b'tiger', b'cat', b'dog', b'dog', b'cat', b'dog',
       b'dog', b'tiger', b'tiger', b'tiger', b'tiger', b'dog', b'cat',
       b'dog', b'cat', b'dog', b'cat', b'dog', b'dog', b'cat', b'cat',
       b'tiger', b'dog', b'dog', b'dog', b'tiger', b'cat', b'dog',
       b'tiger', b'cat', b'cat', b'dog', b'tiger', b'dog', b'tiger',
       b'c

In [35]:
def _serialize_example(f0, f1, f2, f3):
    '''
    1개의 tf.train.Example을 생성한 뒤에 Serialize해서(파일로 출력할 수 있는 상태로 변환) 반환하는 함수
    [매개변수] 
        f0, f1, f2, f3 : 속성값들
    '''
    feature = {
        'feature0' : _int64_feature(f0),
        'feature1' : _float_feature(f1),
        'feature2' : _float_feature(f2),
        'feature3' : _bytes_feature(f3),
    }
    example = tf.train.Example(features = tf.train.Features(feature = feature))
    return example.SerializeToString()

### 출력 처리
- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [36]:
import os
tf_dir = 'sample_tfrecord'
if not os.path.isdir(tf_dir):
    os.mkdir(tf_dir)

In [37]:
tfr_file_path = os.path.join(tf_dir, 'data.tfr') #파일경로 : sample_tfrecord / data.tfr #확장자 : tfr, record, tfrecord
print(tfr_file_path)

sample_tfrecord\data.tfr


In [38]:
#TFRecordWriter 생성 - 출력 Stream 객체
tf_writer = tf.io.TFRecordWriter(tfr_file_path)

In [39]:
#출력 작업
for data in zip(feature0, feature1, feature2, feature3):
    #print(data)
    example_serialized = _serialize_example(bool(data[0]), data[1], data[2], data[3]) #넘파이의 boolean과 파이썬의 boolean은 차이가 있어서 파이썬의 boolean타입으로 변경
    tf_writer.write(example_serialized)
    
tf_writer.close()

# TFRecord파일 읽기 및 역직렬화(Deserialize)
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [40]:
# bytes >> Example
tf.train.Example.FromString(example_serialized) #역직렬화


features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "feature2"
    value {
      float_list {
        value: 1.2264212369918823
      }
    }
  }
  feature {
    key: "feature3"
    value {
      bytes_list {
        value: "cat"
      }
    }
  }
}

In [52]:
def _parse_example(serialized_example):
    '''
    직렬화된(tfrecord에 저장된) example을 받아서 feature들을 추출하여 반환하는 함수
    [매개변수]
        serialized_example : bytes - 값을 추출할 직렬화된 example
    '''
    
    #example에서 읽어온 feature들을 저장할 dictionary
    #'저장할 때 지정한 featurename' : FixedLenFeature(빈 feature)
    features = {
        'feature0' : tf.io.FixedLenFeature([], tf.int64),
        'feature1' : tf.io.FixedLenFeature([], tf.float32),
        'feature2' : tf.io.FixedLenFeature([], tf.float32),
        'feature3' : tf.io.FixedLenFeature([], tf.string)
    }
    
    parsed_example = tf.io.parse_single_example(serialized_example, features)
    
    f0 = tf.cast(parsed_example['feature0'], tf.bool)
    f1 = tf.cast(parsed_example['feature1'], tf.float32)
    f2 = tf.cast(parsed_example['feature2'], tf.float32)
    f3 = tf.cast(parsed_example['feature3'], tf.string)
    
    return f0, f1, f2, f3

In [54]:
# TFRecord 파일에서 읽기 작업
dataset = tf.data.TFRecordDataset(tfr_file_path).map(_parse_example)


In [55]:
for f0, f1, f2, f3 in dataset.take(3):
    print(f0, f1, f2, f3)

tf.Tensor(True, shape=(), dtype=bool) tf.Tensor(3.0, shape=(), dtype=float32) tf.Tensor(-0.47516176, shape=(), dtype=float32) tf.Tensor(b'tiger', shape=(), dtype=string)
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(3.0, shape=(), dtype=float32) tf.Tensor(-0.6182045, shape=(), dtype=float32) tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(3.0, shape=(), dtype=float32) tf.Tensor(-0.070995085, shape=(), dtype=float32) tf.Tensor(b'tiger', shape=(), dtype=string)
